In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
import logging
import sys
import numpy as np
import time
from scipy.io import loadmat
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras import backend as k
import matplotlib.pyplot as plt
from keras.layers import Input, Activation, Add, Dense, Flatten, Concatenate, Dropout, \
    Reshape, Multiply, Lambda, Subtract, ReLU, Conv2DTranspose, LeakyReLU
from keras.layers.convolutional import Conv2D, Conv3D, AveragePooling2D, MaxPooling2D
# from keras_contrib.layers import InstanceNormalization
from InstanceNormalization import InstanceNormalization
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.losses import *
from keras.optimizers import SGD, Adam
from keras.datasets import mnist
import cv2


""" Two points to be noted that cause inconsistency 
between the losses from combined model and standalone model:
1. Batch norm runs in inference mode when trainable=False.
2. Training patch GAN in standalone model with mixed images batch 
will result in batch statistics that is very different from the one of the combined model """


class GAP:
    def __init__(self):
        self.sanitizer = self.build_sanitizer()
        self.d_privacy = self.build_discriminator('d_privacy')
        self.d_utility = load_model("./TrainedModels/d_utility.h5")
        self.regularizer = self.build_regularizer()
        self.posterior = self.build_posterior()

        self.regularizer.compile(optimizer=Adam(0.0001, 0.50),
                                 loss=['binary_crossentropy'],
                                 metrics=['binary_crossentropy'])

        self.d_privacy.compile(optimizer=Adam(0.0002, 0.50),
                               loss=["categorical_crossentropy"],
                               metrics=['accuracy'])

        self.d_privacy.trainable = False
        self.regularizer.trainable = False
        self.d_utility.trainable = False

        int_layer = self.d_utility.layers[2].output
        self.loss_func = Model(self.d_utility.input, int_layer)
        self.loss_func.trainable = False

        for l in self.loss_func.layers:
            l.trainable = False
        self.loss_func.trainable = False

        d_p_out = self.d_privacy(self.sanitizer.output)
        rgl_out = self.regularizer(self.sanitizer.output)
        pst_out = self.posterior(self.sanitizer.output)
        self.combined = Model(self.sanitizer.input, [rgl_out, d_p_out, pst_out])
        self.combined.compile(optimizer=Adam(0.0001, 0.50),
                              loss_weights=[Wx, Wy, Wz],
                              loss=['binary_crossentropy', self.hard_loss, 'mae'],
                              metrics=[['binary_crossentropy'], [self.hard_loss], ['mae']])

    def train(self):
        x_image, y_gender, y_age , IterationImage = self.load_data()
        
        
        
        f = open("v3_Output/v3_logs_NEW/log_" +str(Wx)+"_"+str(Wy)+"_"+str(Wz)+ ".txt", "w")
        
        # self.d_utility.fit(x_image, y_age, epochs=20)
        # self.d_utility.save("./TrainedModels/d_utility.h5")

        # m_idx = np.where(y_gender == 0)[0]
        img_iterator = self.data_generator(x_image, y_gender, y_age, batch_size=60)

        real_label = np.ones((60, 8, 8, 1))
        fake_label = np.zeros((60, 8, 8, 1))
        
        

#         for seq in range(max(iterationArray)+2):
        for seq in range(99999999):
            noise_in = np.random.normal(0, 1, (60, 16, 16, 1))
            img_batch, gender_batch, age_batch = img_iterator.__next__()

            sanitized_images = self.sanitizer.predict([noise_in, img_batch])

            mixed_img = np.vstack((img_batch, sanitized_images))
            mixed_label = np.vstack((real_label, fake_label))
            r_loss = self.regularizer.train_on_batch(mixed_img, mixed_label)

            d_loss = self.d_privacy.train_on_batch(sanitized_images, gender_batch)

            c_loss = self.combined.train_on_batch([noise_in, img_batch], [real_label, gender_batch, img_batch])

            print('\r iter: %d , r_loss:%.4f, r_loss_c:%.4f, p_loss:%.4f, p_loss_c:%.4f, u_loss:%.4f ' %
                  (seq, r_loss[0], c_loss[1], d_loss[0], c_loss[2], c_loss[3]), end='\r')
            
            # save and view iterations intervals.<><><><><><><><><><><><><>
            
            if seq%500 == 0 :
                
                f.write("iter = "+str(seq) + " , r_loss = " + str(r_loss[0]) + " , r_loss_c = " + str(c_loss[1]) + " , p_loss = " + str(d_loss[0]) + " , p_loss_c = " + str(c_loss[2]) + " , u_loss = " + str(c_loss[3]) + "\n")
                print("iter = "+str(seq) + " , r_loss = " + str(r_loss[0]) + " , r_loss_c = " + str(c_loss[1]) + " , p_loss = " + str(d_loss[0]) + " , p_loss_c = " + str(c_loss[2]) + " , u_loss = " + str(c_loss[3]))
                # cv2.imwrite('Pic_'+str(seq)+'.jpg',IterationImage)
                PredictImage = IterationImage.reshape(1,64,64,1)
                PredictImage = PredictImage/255
#                 self.sanitizer.save("./Models/sanitizer.h5")
#                 model = load_model('./Models/sanitizer.h5')
                noise_in_pred = np.random.normal(0, 1, (1, 16, 16, 1))
                score = self.sanitizer.predict([noise_in_pred, PredictImage])
                print ("output shape = "+str(score.shape))
                OUTPUT = score.reshape(64,64)
                print ("output shape after = "+str(OUTPUT.shape))
#                 cv2.imwrite('Output/Output_'+str(Wx)+'_'+str(Wy)+'_'+str(Wz)+'_'+str(seq)+'.jpg',OUTPUT*255)
                cv2.imwrite('v3_Output/v3_'+str(Wx)+'_'+str(Wy)+'_'+str(Wz)+'/'+str(seq)+'.jpg',OUTPUT*255)
#                 cv2.imshow('ori_'+str(seq),IterationImage)

#                 cv2.imshow('image_'+str(seq),OUTPUT)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
                self.sanitizer.save('./v3_Output/v3_'+str(Wx)+'_'+str(Wy)+'_'+str(Wz)+'/'+str(Wx)+'_'+str(Wy)+'_'+str(Wz)+'_'+str(seq)+ '_sanitizer.h5')

            # end here <><><>><><><><><><><><><><><><><><><><><><><><><><><>
            
        f.close()
        print('\n')
        

    @staticmethod
    def hard_loss(y_true, y_pred):
        z = categorical_crossentropy(y_true, y_pred)
        loss = 100*K.square(K.maximum(0.0, 0.5-K.mean(z)))
        return loss

    @staticmethod
    def load_data():
        d = loadmat('./UTKFace_Gray_AgeAbove3.mat')
        image, gender, age = d["image"], d["gender"][0], d["age"][0]

        idx = np.where(age > 0)[0]
        IterationImage = image[idx[1]]
        image = image[idx] / 255.0
        gender = gender[idx]
        gender = np_utils.to_categorical(gender, 2)

        indexes = np.arange(len(image))
        np.random.shuffle(indexes)
        image = image[indexes]
        gender = gender[indexes]
        age = age[indexes]

        return image, gender, age , IterationImage

    @staticmethod
    def data_generator(x, y, z, batch_size=30):
        samples_per_epoch = x.shape[0]
        number_of_batches = samples_per_epoch / batch_size
        counter = 0

        indexes = np.arange(samples_per_epoch)
        np.random.shuffle(indexes)
        x = x[indexes]
        y = y[indexes]
        z = z[indexes]

        while True:
            x_batch = x[batch_size*counter: batch_size*(counter+1)]
            y_batch = y[batch_size*counter: batch_size*(counter+1)]
            z_batch = z[batch_size * counter: batch_size * (counter + 1)]
            counter += 1
            yield x_batch, y_batch, z_batch

            if (counter+1) >= number_of_batches:
                indexes = np.arange(samples_per_epoch)
                np.random.shuffle(indexes)
                x = x[indexes]
                y = y[indexes]
                z = z[indexes]
                counter = 0

    @staticmethod
    def build_sanitizer():
        def resnet_block(x):
            z = Conv2D(filters=128, kernel_size=3, strides=1, padding="same")(x)
            z = BatchNormalization(momentum=0.8)(z)
            z = LeakyReLU(alpha=0.2)(z)

            z = Conv2D(filters=128, kernel_size=3, strides=1, padding="same")(z)
            z = BatchNormalization(momentum=0.8)(z)

            z = Add()([z, x])
            z = LeakyReLU(alpha=0.2)(z)

            return z

        n1 = Input(shape=(16, 16, 1))
        n2 = Conv2D(filters=64,
                    kernel_size=(4, 4),
                    strides=(1, 1),
                    padding='same')(n1)
        n3 = Conv2D(filters=128,
                    kernel_size=(4, 4),
                    strides=(1, 1),
                    padding='same')(n2)

        x1 = Input(shape=(64, 64, 1))
        x2 = Conv2D(filters=32,
                    kernel_size=(6, 6),
                    strides=(1, 1),
                    padding='same')(x1)
        x2 = BatchNormalization(momentum=0.8)(x2)
        x2 = LeakyReLU(alpha=0.2)(x2)

        x3 = Conv2D(filters=64,
                    kernel_size=(3, 3),
                    strides=(2, 2),
                    padding='same')(x2)
        x3 = BatchNormalization(momentum=0.8)(x3)
        x3 = LeakyReLU(alpha=0.2)(x3)

        x3 = Conv2D(filters=128,
                    kernel_size=(3, 3),
                    strides=(2, 2),
                    padding='same')(x3)
        x3 = BatchNormalization(momentum=0.8)(x3)
        x3 = LeakyReLU(alpha=0.2)(x3)

        x4 = resnet_block(x3)
        x4 = Add()([x4, n3])
        x4 = resnet_block(x4)
        x4 = resnet_block(x4)

        x5 = Conv2DTranspose(filters=64,
                             kernel_size=(3, 3),
                             padding='same',
                             strides=(2, 2))(x4)
        x5 = BatchNormalization(momentum=0.8)(x5)
        x5 = LeakyReLU(alpha=0.2)(x5)

        x6 = Conv2DTranspose(filters=1,
                             kernel_size=(3, 3),
                             padding='same',
                             activation='sigmoid',
                             strides=(2, 2))(x5)

        return Model(inputs=[n1, x1], outputs=x6, name='model_sanitizer')

    @staticmethod
    def build_discriminator(name='discriminator'):
        x = Input(shape=(64, 64, 1))

        z = Conv2D(filters=32,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(x)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=64,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=128,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=32,
                   kernel_size=(4, 4),
                   strides=(1, 1),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Flatten()(z)
        y = Dense(units=2, activation="softmax")(z)

        return Model(x, y, name=name)

    @staticmethod
    def build_regularizer():
        x = Input(shape=(64, 64, 1))

        z = Conv2D(filters=32,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(x)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=64,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=128,
                   kernel_size=(4, 4),
                   strides=(2, 2),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        z = Conv2D(filters=32,
                   kernel_size=(4, 4),
                   strides=(1, 1),
                   padding='same')(z)
        z = InstanceNormalization()(z)
        z = LeakyReLU(alpha=0.2)(z)

        y = Conv2D(filters=1,
                   kernel_size=(4, 4),
                   activation='sigmoid',
                   strides=(1, 1),
                   padding='same')(z)

        return Model(x, y, name='regularizer')

    @staticmethod
    def build_posterior():
        x_in = Input(shape=(64, 64, 1))

        x = Conv2D(filters=32,
                   kernel_size=(6, 6),
                   strides=(1, 1),
                   padding='same')(x_in)
        x = BatchNormalization(momentum=0.6)(x)
        x = LeakyReLU(alpha=0.2)(x)

        x = Conv2D(filters=64,
                   kernel_size=(3, 3),
                   strides=(1, 1),
                   padding='same')(x)
        x = BatchNormalization(momentum=0.6)(x)
        x = LeakyReLU(alpha=0.2)(x)

        x = Conv2D(filters=32,
                   kernel_size=(3, 3),
                   strides=(1, 1),
                   padding='same')(x)
        x = BatchNormalization(momentum=0.6)(x)
        x = LeakyReLU(alpha=0.2)(x)

        x = Conv2D(filters=1,
                   kernel_size=(3, 3),
                   strides=(1, 1),
                   activation='sigmoid',
                   padding='same')(x)

        return Model([x_in], x, name='recognizer')

    @staticmethod
    def pixel_loss(y_true, y_pred):
        return K.mean(K.square(y_true - y_pred))

    def perceptual_loss(self, y_true, y_pred):
        return 2*K.mean(K.abs(self.loss_func(y_true) - self.loss_func(y_pred)))


if __name__ == '__main__':
    iterationArray = [10,50,100,150,300,500,800,1200,3000,4500,5000,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,11000,11500,12000,12500,13000,13500,14000,14500,15000,15500,16000,16500,17000,17500,18000,18500,19000,19500,20000,20500,21000,21500,22000,22500,23000,23500,24000,24500,25000,25500,26000,26500,27000,27500,28000,28500,29000,29500,30000] #debug,30500,31000,31500,32000,32500,33000,33500,34000,34500,35000,35500,36000,36500,37000,37500,38000,38500,39000,39500,40000
    
    WeightIterationsX = [1]
    WeightIterationsY = [1]
    WeightIterationsZ = [4]
    
    count = [0,0,0]
    for Wx in WeightIterationsX:
        count[0] += 1
        for Wy in WeightIterationsY:
            count[1] += 1
            for Wz in WeightIterationsZ:
                count[2] += 1
                print("Iteration : Wx = " + str(Wx) + "\nIteration : Wy = " + str(Wy) + "\nIteration : Wz = " + str(Wz))
                print("\nIteration = " + str(count[0]*count[1]*count[2]) + "/100")
                Test = GAP()
                Test.train()










Iteration : Wx = 1
Iteration : Wy = 1
Iteration : Wz = 4

Iteration = 1/100
iter = 0 , r_loss = 0.7825145721435547 , r_loss_c = 0.5461064577102661 , p_loss = 0.8294727206230164 , p_loss_c = 0.0 , u_loss = 0.25619223713874817
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 500 , r_loss = 0.5760357975959778 , r_loss_c = 0.6638498902320862 , p_loss = 0.5353631973266602 , p_loss_c = 0.0 , u_loss = 0.0839034840464592
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 1000 , r_loss = 0.5286567211151123 , r_loss_c = 0.5668789148330688 , p_loss = 0.5966490507125854 , p_loss_c = 0.13569170236587524 , u_loss = 0.09209056198596954
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 1500 , r_loss = 0.6950835585594177 , r_loss_c = 0.7240877151489258 , p_loss = 0.6073187589645386 , p_loss_c = 0.0 , u_loss = 0.07964646071195602
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 2000 , r_loss = 0.5540713667869568 , r_loss_c = 0.76658016

iter = 37000 , r_loss = 0.6513616442680359 , r_loss_c = 1.8771239519119263 , p_loss = 0.5287716388702393 , p_loss_c = 0.6029371619224548 , u_loss = 0.0712665244936943
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 37500 , r_loss = 0.4644937217235565 , r_loss_c = 1.4097497463226318 , p_loss = 0.6174810528755188 , p_loss_c = 0.0 , u_loss = 0.07707912474870682
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 38000 , r_loss = 0.48998770117759705 , r_loss_c = 1.2254670858383179 , p_loss = 0.6694956421852112 , p_loss_c = 0.0 , u_loss = 0.07366111874580383
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 38500 , r_loss = 0.44165435433387756 , r_loss_c = 1.6618435382843018 , p_loss = 0.5453255772590637 , p_loss_c = 0.1450089067220688 , u_loss = 0.07298476248979568
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 39000 , r_loss = 0.4801897704601288 , r_loss_c = 0.993862509727478 , p_loss = 0.5128861665725708 , p_loss_c = 

output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 55500 , r_loss = 0.4576590955257416 , r_loss_c = 1.2544506788253784 , p_loss = 0.522434413433075 , p_loss_c = 0.26204851269721985 , u_loss = 0.08427352458238602
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 56000 , r_loss = 0.39945128560066223 , r_loss_c = 1.6448006629943848 , p_loss = 0.6814759969711304 , p_loss_c = 0.0 , u_loss = 0.07689475268125534
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 56500 , r_loss = 0.47954410314559937 , r_loss_c = 1.7182689905166626 , p_loss = 0.614571213722229 , p_loss_c = 0.0 , u_loss = 0.0754767656326294
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 57000 , r_loss = 0.5506076812744141 , r_loss_c = 1.9167704582214355 , p_loss = 0.6170139312744141 , p_loss_c = 0.0 , u_loss = 0.07706189155578613
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 57500 , r_loss = 0.5193784236907959 , r_loss_c = 2.150748252868652

iter = 74000 , r_loss = 0.39877134561538696 , r_loss_c = 1.4197593927383423 , p_loss = 0.5175734758377075 , p_loss_c = 0.5938783288002014 , u_loss = 0.08171642571687698
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 74500 , r_loss = 0.45792797207832336 , r_loss_c = 2.0043885707855225 , p_loss = 0.45133092999458313 , p_loss_c = 1.3764877319335938 , u_loss = 0.07999546080827713
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 75000 , r_loss = 0.3861601650714874 , r_loss_c = 2.089980363845825 , p_loss = 0.6490305066108704 , p_loss_c = 0.0 , u_loss = 0.08311932533979416
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 75500 , r_loss = 0.48462724685668945 , r_loss_c = 1.0179007053375244 , p_loss = 0.585279107093811 , p_loss_c = 0.3640953600406647 , u_loss = 0.07488643378019333
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 76000 , r_loss = 0.44827398657798767 , r_loss_c = 1.9355523586273193 , p_loss = 0.631087243556

iter = 92500 , r_loss = 0.45029789209365845 , r_loss_c = 1.5370770692825317 , p_loss = 0.5095966458320618 , p_loss_c = 0.18118086457252502 , u_loss = 0.07792246341705322
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 93000 , r_loss = 0.43793532252311707 , r_loss_c = 1.8654093742370605 , p_loss = 0.6418449282646179 , p_loss_c = 0.0 , u_loss = 0.07598521560430527
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 93500 , r_loss = 0.460464209318161 , r_loss_c = 1.418831467628479 , p_loss = 0.49151644110679626 , p_loss_c = 0.1895197033882141 , u_loss = 0.07212379574775696
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 94000 , r_loss = 0.4460020959377289 , r_loss_c = 1.5661948919296265 , p_loss = 0.6067692637443542 , p_loss_c = 0.0 , u_loss = 0.07321403175592422
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 94500 , r_loss = 0.7220458984375 , r_loss_c = 2.2255656719207764 , p_loss = 0.5313695073127747 , p_loss_c = 0

...

iter = 184500 , r_loss = 0.3398820757865906 , r_loss_c = 1.9479831457138062 , p_loss = 0.4375770688056946 , p_loss_c = 0.16384121775627136 , u_loss = 0.0789351835846901
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 185000 , r_loss = 0.1565113216638565 , r_loss_c = 4.352467060089111 , p_loss = 0.6752035021781921 , p_loss_c = 0.0 , u_loss = 0.08427801728248596
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 185500 , r_loss = 0.45956212282180786 , r_loss_c = 5.097084045410156 , p_loss = 0.4461008608341217 , p_loss_c = 0.53970867395401 , u_loss = 0.08623794466257095
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 186000 , r_loss = 0.3542458117008209 , r_loss_c = 0.5199700593948364 , p_loss = 0.5179732441902161 , p_loss_c = 0.1364707350730896 , u_loss = 0.07630350440740585
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 186500 , r_loss = 1.633819580078125 , r_loss_c = 2.008432626724243 , p_loss = 0.528288304805755

iter = 221500 , r_loss = 0.46316927671432495 , r_loss_c = 1.1393280029296875 , p_loss = 0.4826616644859314 , p_loss_c = 0.0 , u_loss = 0.08518211543560028
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 222000 , r_loss = 0.319741815328598 , r_loss_c = 2.9248807430267334 , p_loss = 0.5551372170448303 , p_loss_c = 0.027135271579027176 , u_loss = 0.08101107180118561
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 222500 , r_loss = 0.401329904794693 , r_loss_c = 1.838343858718872 , p_loss = 0.5390676856040955 , p_loss_c = 0.04806482046842575 , u_loss = 0.08505978435277939
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 223000 , r_loss = 0.9394261240959167 , r_loss_c = 2.4891555309295654 , p_loss = 0.5052285194396973 , p_loss_c = 0.0 , u_loss = 0.08469800651073456
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 223500 , r_loss = 0.260464608669281 , r_loss_c = 2.5256004333496094 , p_loss = 0.5720808506011963 , p_loss

iter = 240000 , r_loss = 0.4028793275356293 , r_loss_c = 1.8777912855148315 , p_loss = 0.5846121311187744 , p_loss_c = 0.20515695214271545 , u_loss = 0.07304404675960541
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 240500 , r_loss = 0.36169201135635376 , r_loss_c = 2.3278861045837402 , p_loss = 0.5584259033203125 , p_loss_c = 0.007375350687652826 , u_loss = 0.07725933194160461
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 241000 , r_loss = 0.4450555145740509 , r_loss_c = 2.378566026687622 , p_loss = 0.8249253630638123 , p_loss_c = 0.0 , u_loss = 0.08012760430574417
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 241500 , r_loss = 0.49425145983695984 , r_loss_c = 2.8894970417022705 , p_loss = 0.5930827856063843 , p_loss_c = 0.012463310733437538 , u_loss = 0.07682611793279648
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 242000 , r_loss = 0.33343103528022766 , r_loss_c = 1.8806469440460205 , p_loss = 0.464

iter = 258500 , r_loss = 0.4438091516494751 , r_loss_c = 1.539699673652649 , p_loss = 0.5111474990844727 , p_loss_c = 0.0 , u_loss = 0.07270887494087219
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 259000 , r_loss = 0.3436751365661621 , r_loss_c = 1.991172194480896 , p_loss = 0.6735754013061523 , p_loss_c = 0.0 , u_loss = 0.07409501075744629
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 259500 , r_loss = 0.5650630593299866 , r_loss_c = 3.071608543395996 , p_loss = 0.5346910953521729 , p_loss_c = 0.23977848887443542 , u_loss = 0.07230469584465027
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 260000 , r_loss = 0.43015381693840027 , r_loss_c = 2.1636903285980225 , p_loss = 0.5932725071907043 , p_loss_c = 0.0 , u_loss = 0.07639878243207932
output shape = (1, 64, 64, 1)
output shape after = (64, 64)
iter = 260500 , r_loss = 0.4483957290649414 , r_loss_c = 1.4141548871994019 , p_loss = 0.5809214115142822 , p_loss_c = 0.0 , u_los

KeyboardInterrupt: 